<a href="https://colab.research.google.com/github/Joffreybvn/road-accident-belgium-analysis/blob/master/Road_accident_Belgium_2019_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installs & imports

In [ ]:
# Dataset download
!wget -N https://github.com/Joffreybvn/road-accident-belgium-analysis/blob/master/datasets/raw/TF_ACCIDENTS_2019.xlsx?raw=true
!wget -N https://raw.githubusercontent.com/Joffreybvn/road-accident-belgium-analysis/master/datasets/raw/BELGIUM-Municipalities.geojson

## Packages install

In [ ]:
# Install last Seaborn vesrion
!pip install seaborn --upgrade

In [ ]:
# Geopandas, geoplot and folium install

%%time

try:
  import geopandas as gpd
  import geoplot as gplt
  import folium
  
except:
  !pip install folium
  !pip install git+git://github.com/geopandas/geopandas.git
  !apt install proj-bin libproj-dev libgeos-dev
  !pip install git+git://github.com/ResidentMario/geoplot.git

## Packages imports

In [ ]:
# Allow to print multiple output 
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import geopandas as gpd
import geoplot as gplt
#import folium
#from scipy import stats

from datetime import datetime
import pandas as pd
import numpy  as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns

%matplotlib inline

## Useful functions

In [ ]:
def movecol(df, cols_to_move=[], ref_col='', place='After'):
    
    cols = df.columns.tolist()
    if place == 'After':
        seg1 = cols[:list(cols).index(ref_col) + 1]
        seg2 = cols_to_move
    if place == 'Before':
        seg1 = cols[:list(cols).index(ref_col)]
        seg2 = cols_to_move + [ref_col]
    
    seg1 = [i for i in seg1 if i not in seg2]
    seg3 = [i for i in cols if i not in seg1 + seg2]
    
    return(df[seg1 + seg2 + seg3])

## Datasets creation

In [ ]:
df = pd.read_excel('/content/TF_ACCIDENTS_2019.xlsx?raw=true')
# map_df = gpd.read_file('/content/BELGIUM-Municipalities.geojson')

# Dataset general cleaning

In [ ]:
df.head()
df.shape

It seems we have some duplicated columns: In french and in deutch. And some useless geolocalisation columns. Remove them:

In [ ]:
df.drop(columns=['CD_DAY_OF_WEEK',
                 'TX_DAY_OF_WEEK_DESCR_FR',
                 'TX_DAY_OF_WEEK_DESCR_NL',
                 'CD_BUILD_UP_AREA',
                 'CD_COLL_TYPE',
                 'TX_BUILD_UP_AREA_DESCR_NL',
                 'TX_COLL_TYPE_DESCR_NL',
                 'CD_LIGHT_COND',
                 'TX_LIGHT_COND_DESCR_NL',
                 'CD_ROAD_TYPE',
                 'TX_ROAD_TYPE_DESCR_NL',
                 'TX_MUNTY_DESCR_NL',
                 'CD_DSTR_REFNIS',
                 'TX_ADM_DSTR_DESCR_NL',
                 'TX_ADM_DSTR_DESCR_FR',
                 'TX_PROV_DESCR_NL',
                 'TX_RGN_DESCR_NL'], inplace=True)

In [ ]:
df.head(3)

## Date creation

Some hours are set to -1, some others are on 24.

In [ ]:
df['DT_HOUR'].unique()

Check how many are to -1 and drop them:

In [ ]:
len(df.loc[df['DT_HOUR'] == -1,:])
df = df[df['DT_HOUR'] != -1]

Hours can go from 0 to 23. Change entries with 24 to 23

In [ ]:
df.loc[df['DT_HOUR'] == 24, 'DT_HOUR'] = 23

Create the datetime field with date and hours:

In [ ]:
# Create the 'date' field
df_date = df[['DT_DAY', 'DT_HOUR']].astype(str)
df['datetime'] = pd.to_datetime(df_date.agg('-'.join, axis=1), format='%Y-%m-%d-%H')

# Create the week_day fiel
df['week_day'] = df['datetime'].dt.day_name()

# Drop the DT_DAY & DT_HOUR:
df.drop(columns=['DT_DAY', 'DT_HOUR'], inplace=True)

Create the date field

In [ ]:
#df['date'] = df['datetime'].apply(lambda x: datetime(year=x.year, month=x.month, day=x.day))	
df.set_index(df["datetime"], inplace=True)

# Set as index
df.head(3)

## Renaming the columns

In [ ]:
df.rename(columns = {'TX_DAY_OF_WEEK_DESCR_FR':'week_day',
                     'TX_BUILD_UP_AREA_DESCR_FR':'incident_place',
                     'TX_COLL_TYPE_DESCR_FR':'collision_type',
                     'TX_LIGHT_COND_DESCR_FR': 'light_condition',
                     'TX_ROAD_TYPE_DESCR_FR': 'road_type',
                     'CD_MUNTY_REFNIS': 'municipality_ins',
                     'TX_MUNTY_DESCR_FR': 'municipality_name',
                     'CD_PROV_REFNIS': 'province_ins',
                     'TX_PROV_DESCR_FR': 'province_name',
                     'CD_RGN_REFNIS': 'region_ins',
                     'TX_RGN_DESCR_FR': 'region_name',
                     'MS_ACCT': 'dead_injured',
                     'MS_ACCT_WITH_DEAD': 'dead_directly',
                     'MS_ACCT_WITH_DEAD_30_DAYS': 'dead_total_30_days',
                     'MS_ACCT_WITH_MORY_INJ': 'injured_mortally',
                     'MS_ACCT_WITH_SERLY_INJ': 'injured_severely',
                     'MS_ACCT_WITH_SLY_INJ': 'injured_sightly'}, inplace = True) 

In [ ]:
df.head(3)

Reordering the columns

In [ ]:
# Put the date as first column
df = movecol(df, cols_to_move=['week_day'], ref_col='incident_place', place='Before')

df = movecol(df, cols_to_move=['light_condition',
                               'road_type',
                               'incident_place',
                               'collision_type',
                               'dead_injured',
                               'dead_total_30_days',
                               'dead_directly',
                               'injured_mortally',
                               'injured_severely',
                               'injured_sightly'], ref_col='week_day', place='After')

In [ ]:
df.head()
